<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [516]:
# !pip3 install dash
# !pip3 install jupyter_dash
# !pip3 install dash_bootstrap_components

In [517]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import jupyter_dash
from dash import Dash, html, dcc, no_update
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objs as go
from datetime import date
from datetime import datetime

In [518]:
unis = pd.read_csv('World_University_Rankings.csv')

In [519]:
unis[:100]

,University,Country,City,Region,Logo,Link,Type,Size,Faculty_count,International_Students,Students_count,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Latitude,Longitude
0,University Paris-Saclay,France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,8010,7649,40050,241,0,0,0,0,0,48.713651,2.203623
1,Baku Engineering University,Azerbaijan,Baku,Asia,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/b...,Private,S,225,78,4725,703,0,0,0,0,0,40.473905,49.727411
2,"Ecole normale supérieure, Paris",France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/e...,Public,S,178,374,1958,33,43,0,0,0,0,48.841707,2.344792
3,Université Pierre et Marie Curie (UPMC),France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,4680,8782,42120,141,131,0,0,0,0,48.847301,2.357596
4,Université Paris-Sorbonne (Paris IV),France,Paris,Europe,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,4680,8782,42120,221,293,0,0,0,0,48.849412,2.344538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The University of Tokyo,Japan,Tokyo,Asia,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,L,4473,3983,26838,34,28,23,22,24,23,35.711164,139.760562
96,University of Michigan-Ann Arbor,United States,Ann Arbor,North America,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,7132,7341,42792,23,21,20,21,21,23,42.278030,-83.736564
97,Johns Hopkins University,United States,Baltimore,North America,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/j...,Private,L,4855,5070,14565,17,17,21,24,25,25,39.329808,-76.620723
98,University of Toronto,Canada,Toronto,North America,https://www.topuniversities.com/sites/default/...,https://www.topuniversities.com/universities/u...,Public,XL,9881,27536,79048,32,31,28,29,25,26,43.663462,-79.397760


In [520]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [521]:
# x = np.logical_and(unis['Country']=='Russia', unis['Rank_2022']>0)
# fig = px.scatter_mapbox(
#         unis[x].iloc[:100,:], lon='Longitude', lat='Latitude', color='Rank_2022', hover_name='University', size_max=40, hover_data=['Country'], zoom=3)
# fig.update_layout(mapbox_style="carto-positron")
# fig.show()

In [522]:
app.layout = dbc.Container(children=[
    dbc.Row([
        dbc.Col(dcc.Graph(id='map', figure={}), width=8),
        dbc.Col(html.Div(id='table', style={'maxHeight': '350px', 'overflowY': 'scroll'}))
    ]),

    dbc.Row([
        dbc.Col(dcc.Slider(id='mapSlider',
                  min=2017,
                  max=2022,
                  value=2022,
                  step=None,
                  marks={i: str(i) for i in range(2017, 2023)}
                  ), width=6),
             
        dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='map_region'), width=3),
        dbc.Col(dcc.Dropdown(placeholder='Select Country', id='map_country')),
    ])
])

In [523]:
@app.callback(
    Output('map_country', 'options'),
    Output('map', 'figure'),
    Output("table", "children"),
    Input('mapSlider', 'value'),
    Input('map_region', 'value'),
    Input('map_country', 'value')
)


def updateMap(year, region, country):
    year = 'Rank_' + str(year)
    df = unis[unis[year] > 0].sort_values(year, axis=0)
    print
    zoom = 0.6

    if region:
        zoom = 1.8
        df = df[df['Region'] == region]

    if country:
        zoom = 3
        df = df[df['Country'] == country]

    # token = 'pk.eyJ1IjoiYWZuYW5ldG1hbiIsImEiOiJjbDJqN3F5cW0wMHprM2VwazQ0aHdvZDBhIn0.X032SoU1tft9zgusUaEqSQ'
    # fig = go.Figure(go.Scattermapbox(lon=df[:100]['Longitude'],
    #                                  lat=df[:100]['Latitude'],
    #                                  mode='markers',
    #                                  marker = {'size': 10, 'symbol': 'college'},
    #                                  ))
    # fig.update_layout(mapbox = {'accesstoken': token, 'style': "outdoors", 'zoom': zoom})

    # px.set_mapbox_access_token('pk.eyJ1IjoiYWZuYW5ldG1hbiIsImEiOiJjbDJqN3F5cW0wMHprM2VwazQ0aHdvZDBhIn0.X032SoU1tft9zgusUaEqSQ')
    fig = px.scatter_mapbox(df[:100],
                            lon='Longitude',
                            lat='Latitude',
                            color=year,
                            hover_name='University',
                            hover_data=['Country'],
                            zoom=zoom,
                            title = 'Ranking of Universities Around the World',
                            mapbox_style='carto-positron')
    
    fig.update_traces(marker = {'size':10, 'opacity':0.5}, selector={'type': 'scattermapbox'})

    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    table = dbc.Table.from_dataframe(df[[year, 'University']], striped=True, bordered=True, hover=True, responsive=True)
    
    return countries, fig, table

In [524]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [525]:
# from jupyter_dash import JupyterDash
# from dash import Dash, dcc, html, Input, Output, no_update
# import plotly.graph_objects as go
# import pandas as pd

# ## create sample random data
# df = pd.DataFrame({
#     'x': [80,2,10],
#     'y': [20,3,25],
#     'color': ['red','green','blue'],
#     'img_url': [
#         "https://upload.wikimedia.org/wikipedia/commons/0/03/Refugee_care_near_Poland_border_train_station_20220228.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/4/45/Ukrainian_refugees_from_2022%2C_crossing_into_Poland.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/b/b2/Warsaw_Central_Station_during_Ukrainian_refugee_crisis_10.jpg"
        
#     ]
# })

# fig = go.Figure(data=[
#     go.Scatter(
#         x=df['x'], 
#         y=df['y'], 
#         mode='markers',
#         marker=dict(color=df['color'])
#     )
# ])


# fig.update_traces(hoverinfo="none", hovertemplate=None)

# app = JupyterDash(_name_)

# server = app.server

# app.layout = html.Div([
#     dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
#     dcc.Tooltip(id="graph-tooltip", direction='bottom'),
# ])


# @app.callback(
#     Output("graph-tooltip", "show"),
#     Output("graph-tooltip", "bbox"),
#     Output("graph-tooltip", "children"),
#     Output("graph-tooltip", "direction"),
    
#     Input("graph-basic-2", "hoverData"),
# )
# def display_hover(hoverData):
#     if hoverData is None:
#         return False, no_update, no_update ,no_update

#     # demo only shows the first point, but other points may also be available
#     pt = hoverData["points"][0]
#     bbox = pt["bbox"]
#     num = pt["pointNumber"]
    
    
    
#     df_row = df.iloc[num]
#     img_src = df_row['img_url']

#     children = [
#         html.Img(
#             src=img_src,
#             style={"width": "250px"},
#         ),
#         html.P("ukrain refugees"),
#     ]
    
#     y = hoverData["points"][0]['y']
#     direction = "bottom" if y > 2 else "top"
    
#     return True, bbox, children , direction

# app.run_server(mode="inline")